In [1]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 199 kB 33.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ad0401e40d89647a11ce05d713f7911600748ac23b3fe71b419c2499952dd0d8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf


sc=pyspark.SparkContext()
# spark = SparkSession.builder.getOrCreate()

spark = SparkSession\
    .builder\
    .config("spark.driver.memory", "16g")\
    .config("spark.executor.memory", "4g")\
    .getOrCreate()


import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

train_data = spark.read.csv("/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/train_data.csv", header=True, inferSchema=True)
test_data = spark.read.csv("/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/test_data.csv", header=True, inferSchema=True)


Mounted at /content/drive


# RandomOverSampling 알고리즘 구현

In [4]:
# 주 클래스와 소수 클래스의 비율 계산
origin_major_df = train_data.filter(train_data.HeartDisease_Index == 0.0)
origin_minor_df = train_data.filter(train_data.HeartDisease_Index == 1.0)

ratio = int(origin_major_df.count()/origin_minor_df.count())
print("ratio: {}".format(ratio))
a = range(ratio)

# 소수 클래스 복제
major_df = train_data.filter(train_data.HeartDisease_Index == 0.0)
minor_df = train_data.filter(train_data.HeartDisease_Index == 1.0)

create_data= minor_df.withColumn("dummy", F.explode(F.array([F.lit(x) for x in a]))).drop('dummy')

# 다수 클래스와 병합
oversample_train_data = major_df.unionAll(create_data)

oversample_train_data.show()

oversample_major_df = oversample_train_data.filter(oversample_train_data.HeartDisease_Index == 0.0)
oversample_minor_df = oversample_train_data.filter(oversample_train_data.HeartDisease_Index == 1.0)

# 데이터 비율 및 수
print("샘플링 전 데이터 개수: {}".format(train_data.count()))
print("샘플링 전 0인 경우 : {}".format(major_df.count()))
print("샘플링 전 1인 경우 : {}".format(minor_df.count()))
print()
print()
print("오버 샘플링 후 데이터 전체수: {}".format(oversample_train_data.count()))
print("오버 샘플링 데이터 label 0인 경우 : {}".format(oversample_major_df.count()))
print("오버 샘플링 데이터 label 1인 경우 : {}".format(oversample_minor_df.count()))

ratio: 10
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|HeartDisease_Index|Smoking_Index|AlcoholDrinking_Index|Stroke_Index|DiffWalking_Index|Sex_Index|PhysicalActivity_Index|Asthma_Index|KidneyDisease_Index|SkinCancer_Index|AgeCategory_Index|GenHealth_Index|Diabetic_Index|           BMI_Log|PhysicalHealth_Log|  MentalHealth_Log|     SleepTime_Log|
+------------------+-------------+---------------------+------------+-----------------+---------+----------------------+------------+-------------------+----------------+-----------------+---------------+--------------+------------------+------------------+------------------+------------------+
|               0.0|          0.0|                  0.0|         0.0|              0.0|      0.0|     

# RandomOverSampling으로 데이터 증강한 결과 내보내기

In [5]:
oversample_train_data.toPandas().to_csv('/content/drive/MyDrive/빅데이터 처리 및 응용/팀플/oversample_train_data.csv', index=False, encoding='CP949')